In [88]:
import joblib
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

def preprocessamento_dados(df_vendas):

    # 1. Verificação inicial dos dados
    print("\n=== VERIFICAÇÃO INICIAL ===")
    print(f"Total de registros originais: {len(df_vendas)}")
    print(f"Colunas disponíveis: {list(df_vendas.columns)}")

    # 2. Seleção de colunas relevantes para o projeto de previsão de demanda
    colunas_relevantes = [
        'codigo_produto',           # Identificador do produto
        'data_emissao',             # Data da venda (para features temporais)
        'quantidade',               # Quantidade vendida (variável alvo)
        'total',                    # Valor total da venda
        'custo_medio',              # Custo médio do produto
        'valor_desconto',           # Valor de desconto aplicado
        'codigo_empresa',           # Empresa (pode ser relevante como feature categórica)
        'tipo_pedido'               # Tipo de pedido (feature categórica)
    ]

    # Filtrar apenas colunas relevantes (evitar vazamento de dados)
    df = df_vendas[colunas_relevantes].copy()

    # 3. Limpeza de dados
    print("\n=== LIMPEZA DE DADOS ===")
    # Remover linhas com valores nulos nas colunas críticas
    antes = len(df)
    df.dropna(subset=['quantidade', 'codigo_produto', 'data_emissao'], inplace=True)
    print(f"Registros removidos por valores nulos: {antes - len(df)}")

    # Converter data para formato datetime
    df['data_emissao'] = pd.to_datetime(df['data_emissao'])

    # Converter tipo_pedido para minúsculo
    df['tipo_pedido'] = df['tipo_pedido'].str.lower()

    # 4. Feature Engineering para previsão de demanda
    print("\n=== FEATURE ENGINEERING ===")
    # Extrair características temporais
    df['dia_semana'] = df['data_emissao'].dt.dayofweek  # 0=Segunda, 6=Domingo
    df['mes'] = df['data_emissao'].dt.month
    df['trimestre'] = df['data_emissao'].dt.quarter

    # Calcular média móvel de vendas por produto (útil para modelos de série temporal)
    df.sort_values(['codigo_produto', 'data_emissao'], inplace=True)
    df['media_movel_3meses'] = df.groupby('codigo_produto')['quantidade'].transform(
        lambda x: x.rolling(window=90, min_periods=1).mean()
    )

    # 5. Codificação de variáveis categóricas
    print("\n=== CODIFICAÇÃO CATEGÓRICA ===")
    # One-Hot Encoding para tipo_pedido
    df = pd.get_dummies(df, columns=['tipo_pedido'], prefix='tipo')

    # Label Encoding para codigo_empresa
    df['codigo_empresa'] = df['codigo_empresa'].astype('category').cat.codes

    # 6. Separação em treino e teste ANTES da normalização (evitar vazamento)
    print("\n=== DIVISÃO TREINO/TESTE ===")
    X = df.drop(columns=['quantidade', 'data_emissao'])  # Features
    y = df['quantidade']  # Variável alvo (demanda)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=False
    )
    print(f"Tamanho do treino: {len(X_train)}")
    print(f"Tamanho do teste: {len(X_test)}")

    # 7. Normalização/Padronização (apenas nas colunas numéricas contínuas)
    print("\n=== NORMALIZAÇÃO/PADRONIZAÇÃO ===")
    colunas_numericas = ['total', 'custo_medio', 'valor_desconto', 'media_movel_3meses']

    # Normalização Min-Max (preserva a distribuição original)
    scaler_minmax = MinMaxScaler()
    X_train_minmax = X_train.copy()
    X_train_minmax[colunas_numericas] = scaler_minmax.fit_transform(X_train[colunas_numericas])

    X_test_minmax = X_test.copy()
    X_test_minmax[colunas_numericas] = scaler_minmax.transform(X_test[colunas_numericas])

    # Padronização StandardScaler (útil para modelos sensíveis à escala)
    scaler_standard = StandardScaler()
    X_train_standard = X_train.copy()
    X_train_standard[colunas_numericas] = scaler_standard.fit_transform(X_train[colunas_numericas])

    X_test_standard = X_test.copy()
    X_test_standard[colunas_numericas] = scaler_standard.transform(X_test[colunas_numericas])

    # 8. Exibir resultados da padronização/normalização
    print("\n=== RESULTADOS DA NORMALIZAÇÃO (MinMaxScaler) ===")
    print("Dados de TREINO após normalização:")
    print(X_train_minmax[colunas_numericas].describe().round(2))

    print("\nDados de TESTE após normalização:")
    print(X_test_minmax[colunas_numericas].describe().round(2))

    print("\n=== RESULTADOS DA PADRONIZAÇÃO (StandardScaler) ===")
    print("Dados de TREINO após padronização:")
    print(X_train_standard[colunas_numericas].describe().round(2))

    print("\nDados de TESTE após padronização:")
    print(X_test_standard[colunas_numericas].describe().round(2))

    # 9. Preparação dos dados para salvar
    dados_processados = {
        # Dados originais (apenas para referência)
        'df_original': df,

        # Dados para modelagem
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,

        # Versões normalizadas
        'X_train_minmax': X_train_minmax,
        'X_test_minmax': X_test_minmax,

        # Versões padronizadas
        'X_train_standard': X_train_standard,
        'X_test_standard': X_test_standard,

        # Scalers (para aplicar em novos dados)
        'scaler_minmax': scaler_minmax,
        'scaler_standard': scaler_standard,

        # Informações sobre as transformações
        'colunas_numericas': colunas_numericas,
        'metadata': {
            'projeto': 'Previsão de Demanda - UNIGEX',
            'versao': '1.0',
            'data_processamento': pd.Timestamp.now().strftime('%Y-%m-%d')
        }
    }

    print("\n=== PRÉ-PROCESSAMENTO CONCLUÍDO ===")
    return dados_processados

# Exemplo de uso:
if __name__ == "__main__":
    df_vendas = joblib.load('df_vendas.z')

    # Aplicar pré-processamento
    dados_processados = preprocessamento_dados(df_vendas)

    # Salvar resultados
    joblib.dump(dados_processados, 'padronizado_vendas.z')
    print("Dados processados salvos em 'padronizado_vendas.z'")


=== VERIFICAÇÃO INICIAL ===
Total de registros originais: 10418
Colunas disponíveis: ['codigo_empresa', 'tipo_pedido', 'centro_resultado', 'documento', 'codigo_cliente', 'data_emissao', 'hora_emissao', 'data_fechamento', 'hora_fechamento', 'data_faturamento', 'codigo_produto', 'quantidade', 'total', 'custo_medio', 'valor_desconto']

=== LIMPEZA DE DADOS ===
Registros removidos por valores nulos: 0

=== FEATURE ENGINEERING ===

=== CODIFICAÇÃO CATEGÓRICA ===

=== DIVISÃO TREINO/TESTE ===
Tamanho do treino: 8334
Tamanho do teste: 2084

=== NORMALIZAÇÃO/PADRONIZAÇÃO ===

=== RESULTADOS DA NORMALIZAÇÃO (MinMaxScaler) ===
Dados de TREINO após normalização:
        total   custo_medio  valor_desconto  media_movel_3meses
count  8334.00    8334.00        8334.00           8334.00     
mean      0.11       0.16           0.02              0.10     
std       0.02       0.03           0.06              0.05     
min       0.00       0.00           0.00              0.00     
25%       0.10     

In [95]:
import joblib
import pandas as pd
from IPython.display import display

def display_table(title, dataframe, max_rows=5):
    """Função para exibir tabelas de forma formatada"""
    print("\n" + "="*50)
    print(title)
    print("-"*50)
    print(f"Formato: {dataframe.shape} | Colunas: {list(dataframe.columns)}")
    print("-"*50)
    display(dataframe.head(max_rows))

def main():
    # Carregar os dados
    try:
        dados = joblib.load('padronizado_vendas.z')
        print("Arquivo carregado com sucesso!")
    except Exception as e:
        print(f"Erro ao carregar arquivo: {str(e)}")
        return

    # Configurações de exibição
    pd.set_option('display.max_rows', 100)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    pd.set_option('display.float_format', '{:.4f}'.format)

    # 1. Visualizar estrutura dos dados
    print("\n" + "="*50)
    print("ESTRUTURA DOS DADOS")
    print("-"*50)
    print("Chaves disponíveis:", list(dados.keys()))

    # 2. Visualizar dados originais
    if 'df_original' in dados:
        df_original = dados['df_original']
        # Aplicar filtros para visualização
        df_display = df_original.drop(columns=['codigo_cliente', 'documento'], errors='ignore')
        if 'tipo_pedido' in df_display:
            df_display['tipo_pedido'] = df_display['tipo_pedido'].str.lower()

        display_table("DADOS ORIGINAIS (PRÉ-PROCESSADOS)", df_display)

    # 3. Visualizar dados de treino
    if 'X_train' in dados:
        display_table("DADOS DE TREINO (X_train)", dados['X_train'])

    # 4. Visualizar dados padronizados
    if 'X_train_standard' in dados and 'colunas_numericas' in dados:
        display_table("DADOS PADRONIZADOS (StandardScaler)",
                     dados['X_train_standard'][dados['colunas_numericas']])

    # 5. Visualizar dados normalizados
    if 'X_train_minmax' in dados and 'colunas_numericas' in dados:
        display_table("DADOS NORMALIZADOS (MinMaxScaler)",
                     dados['X_train_minmax'][dados['colunas_numericas']])

    # 6. Visualizar variável alvo
    if 'y_train' in dados:
        display_table("VARIÁVEL ALVO (y_train)", pd.DataFrame(dados['y_train'], columns=['quantidade']))

    # 7. Visualizar metadados
    if 'metadata' in dados:
        print("\n" + "="*50)
        print("METADADOS")
        print("-"*50)
        for key, value in dados['metadata'].items():
            print(f"{key}: {value}")

if __name__ == "__main__":
    main()

Arquivo carregado com sucesso!

ESTRUTURA DOS DADOS
--------------------------------------------------
Chaves disponíveis: ['df_original', 'X_train', 'X_test', 'y_train', 'y_test', 'X_train_minmax', 'X_test_minmax', 'X_train_standard', 'X_test_standard', 'scaler_minmax', 'scaler_standard', 'colunas_numericas', 'metadata']

DADOS ORIGINAIS (PRÉ-PROCESSADOS)
--------------------------------------------------
Formato: (10418, 14) | Colunas: ['codigo_produto', 'data_emissao', 'quantidade', 'total', 'custo_medio', 'valor_desconto', 'codigo_empresa', 'dia_semana', 'mes', 'trimestre', 'media_movel_3meses', 'tipo_dv', 'tipo_oa', 'tipo_pv']
--------------------------------------------------


,codigo_produto,data_emissao,quantidade,total,custo_medio,valor_desconto,codigo_empresa,dia_semana,mes,trimestre,media_movel_3meses,tipo_dv,tipo_oa,tipo_pv
10414,4,2025-02-17,1.0000,123.2000,44.1800,0.0000,2,0,2,1,1.0000,False,True,False
10415,5,2025-02-17,2.0000,430.9600,67.7800,0.0000,2,0,2,1,2.0000,False,True,False
10416,6,2025-02-17,3.0000,471.6300,63.7000,0.0000,2,0,2,1,3.0000,False,True,False
2598,12,2023-01-02,1.0000,16.0000,8.2955,2.0000,1,0,1,1,1.0000,False,False,True
2927,12,2023-01-05,1.0000,20.0000,8.2956,0.0000,1,3,1,1,1.0000,False,False,True



DADOS DE TREINO (X_train)
--------------------------------------------------
Formato: (8334, 12) | Colunas: ['codigo_produto', 'total', 'custo_medio', 'valor_desconto', 'codigo_empresa', 'dia_semana', 'mes', 'trimestre', 'media_movel_3meses', 'tipo_dv', 'tipo_oa', 'tipo_pv']
--------------------------------------------------


,codigo_produto,total,custo_medio,valor_desconto,codigo_empresa,dia_semana,mes,trimestre,media_movel_3meses,tipo_dv,tipo_oa,tipo_pv
10414,4,123.2000,44.1800,0.0000,2,0,2,1,1.0000,False,True,False
10415,5,430.9600,67.7800,0.0000,2,0,2,1,2.0000,False,True,False
10416,6,471.6300,63.7000,0.0000,2,0,2,1,3.0000,False,True,False
2598,12,16.0000,8.2955,2.0000,1,0,1,1,1.0000,False,False,True
2927,12,20.0000,8.2956,0.0000,1,3,1,1,1.0000,False,False,True



DADOS PADRONIZADOS (StandardScaler)
--------------------------------------------------
Formato: (8334, 4) | Colunas: ['total', 'custo_medio', 'valor_desconto', 'media_movel_3meses']
--------------------------------------------------


...


DADOS NORMALIZADOS (MinMaxScaler)
--------------------------------------------------
Formato: (8334, 4) | Colunas: ['total', 'custo_medio', 'valor_desconto', 'media_movel_3meses']
--------------------------------------------------


,total,custo_medio,valor_desconto,media_movel_3meses
10414,0.1035,0.1559,0.0000,0.0648
10415,0.1161,0.1602,0.0000,0.0741
10416,0.1177,0.1595,0.0000,0.0833
2598,0.0992,0.1494,0.0007,0.0648
2927,0.0993,0.1494,0.0000,0.0648



VARIÁVEL ALVO (y_train)
--------------------------------------------------
Formato: (8334, 1) | Colunas: ['quantidade']
--------------------------------------------------


,quantidade
10414,1.0000
10415,2.0000
10416,3.0000
2598,1.0000
2927,1.0000



METADADOS
--------------------------------------------------
projeto: Previsão de Demanda - UNIGEX
versao: 1.0
data_processamento: 2025-05-26
